In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Primero tenemos que conectarnos a la base de datos
conn = psycopg2.connect(
    dbname="hoteles_eventos_madrid",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

In [3]:
# Creamos un cursor que nos permita ejecutar consultas a esta conexión que acabamos de hacer
cur = conn.cursor()

In [4]:
# Comprobamos la conexión
cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit',)


In [5]:
# Cargamos los archivos necesarios para la tabla ciudad
df_hoteles = pd.read_pickle("../data/data_transform/reservas_hoteles_limpio.pkl")
df_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 5171
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   nombre           15000 non-null  object        
 2   apellido         15000 non-null  object        
 3   mail             15000 non-null  object        
 4   competencia      15000 non-null  bool          
 5   fecha_reserva    15000 non-null  datetime64[ns]
 6   inicio_estancia  15000 non-null  datetime64[ns]
 7   final_estancia   15000 non-null  datetime64[ns]
 8   id_hotel         15000 non-null  int64         
 9   nombre_hotel     15000 non-null  object        
 10  valoracion       15000 non-null  float64       
 11  nombre_ciudad    15000 non-null  object        
 12  precio_noche     15000 non-null  float64       
 13  id_cliente       15000 non-null  object        
 14  id_ciudad        15000 non-null  int64      

In [6]:
tabla_ciudad = df_hoteles[["id_ciudad", "nombre_ciudad", ]]
tabla_ciudad.head()

,id_ciudad,nombre_ciudad
0,1,Madrid
1,1,Madrid
2,1,Madrid
3,1,Madrid
4,1,Madrid


In [7]:
# Primero tenemos que conectarnos a la base de datos
conn = psycopg2.connect(
    dbname="hoteles_eventos_madrid",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
# Creamos un cursor que nos permita ejecutar consultas a esta conexión que acabamos de hacer
cur = conn.cursor()

In [8]:
# Creamos la consulta
insert_query = """
    INSERT INTO ciudad (id_ciudad, nombre_ciudad)
    VALUES (%s, %s)
"""

In [9]:
# Creamos un conjunto para eliminar duplicados
data_to_insert = list(set(
    (row['id_ciudad'], row['nombre_ciudad']) 
    for _, row in tabla_ciudad.iterrows()
))

# Mostrar las primeras 3 filas
data_to_insert[:3]

[(1, 'Madrid')]

In [10]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
conn.commit()
# Comprobamos en la base de datos

In [11]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM ciudad LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(1, 'Madrid')]

In [12]:
# Cargamos el archivo de eventos que viene de la API
df_eventos = pd.read_pickle("../data/data_raw/eventos_madrid.pkl")
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  295 non-null    object        
 1   url_evento     295 non-null    object        
 2   codigo_postal  295 non-null    int64         
 3   direccion      295 non-null    object        
 4   horario        295 non-null    object        
 5   organizacion   295 non-null    object        
 6   fecha_inicio   295 non-null    datetime64[ns]
 7   fecha_fin      295 non-null    datetime64[ns]
 8   nombre_ciudad  295 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 20.9+ KB


In [13]:
tabla_eventos = df_eventos[["nombre_evento", "url_evento", "codigo_postal", "direccion", "horario",
                             "organizacion", "fecha_inicio", "fecha_fin", "nombre_ciudad"]]
tabla_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,nombre_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,"CALLE SAN JUSTO 5, Palacio, Centro",,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,"PASEO COLOMBIA 1, LosJeronimos, Retiro",,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,"PLAZA LEGAZPI 8, Chopera, Arganzuela",,Matadero Madrid,2024-10-01,2025-06-01,Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,No disponible,,No disponible,2024-10-14,2025-06-30,Madrid
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,"AVENIDA MONFORTE DE LEMOS 38, ElPilar, Fuencar...",17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid


In [14]:
cur.execute("SELECT id_ciudad, nombre_ciudad FROM ciudad")  
ciudad_list = cur.fetchall()  # Lista de tuplas
ciudad_dict = {nombre.strip().lower(): id_c for id_c, nombre in ciudad_list}
print(ciudad_dict)  # Verificar que los datos se extrajeron correctamente

{'madrid': 1}


In [15]:
data_to_insert = []
for _, row in df_eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"]
    horario = row["horario"]
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"]
    nombre_ciudad = row["nombre_ciudad"]
    id_ciudad = ciudad_dict.get(nombre_ciudad.strip().lower(), None)
    
    data_to_insert.append([nombre_evento, url_evento, codigo_postal, direccion, horario, 
                            fecha_inicio, fecha_fin, organizacion, id_ciudad])

In [16]:
insert_query = """
                INSERT INTO eventos(nombre_evento, url_evento, codigo_postal, direccion, 
                 horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [17]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [18]:
# Comprobamos en la base de datos
query_comprobacion = """
                SELECT * FROM eventos LIMIT 5
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(1,
  '25º aniversario de la revista La Fragua',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD',
  28005,
  'CALLE SAN JUSTO 5, Palacio, Centro',
  '',
  datetime.date(2025, 2, 24),
  datetime.date(2025, 3, 2),
  'Biblioteca Pública Municipal Iván de Vargas (Centro)',
  1),
 (2,
  '60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1, LosJeronimos, Retiro',
  '',
  datetime.date(2025, 2, 27),
  datetime.date(2025, 3, 23),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 (3,
  'A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8, Chopera, Arganzuela',
  '',
  datetime.date(202

In [19]:
tabla_hoteles = df_hoteles[["nombre_hotel", "competencia", "valoracion", "nombre_ciudad" ]]
tabla_hoteles.head()

,nombre_hotel,competencia,valoracion,nombre_ciudad
0,Hotel Monte Verde,False,1.0,Madrid
1,Hotel Brisas del Mar,False,5.0,Madrid
2,Hotel Camino del Sol,False,1.0,Madrid
3,Hotel Puerta del Cielo,False,5.0,Madrid
4,Hotel Encanto Real,False,1.0,Madrid


In [20]:
tabla_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 5171
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nombre_hotel   15000 non-null  object 
 1   competencia    15000 non-null  bool   
 2   valoracion     15000 non-null  float64
 3   nombre_ciudad  15000 non-null  object 
dtypes: bool(1), float64(1), object(2)
memory usage: 483.4+ KB


In [21]:
data_to_insert = []
for _, row in df_hoteles.iterrows():
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = row["valoracion"]
    nombre_ciudad = row["nombre_ciudad"]
    id_ciudad = ciudad_dict.get(nombre_ciudad.strip().lower(), None)
    
    data_to_insert.append([nombre_hotel, competencia, valoracion, id_ciudad])

In [22]:
insert_query = """
                INSERT INTO hoteles(nombre_hotel, competencia, valoracion, id_ciudad)
                VALUES (%s, %s, %s, %s)
"""

In [23]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [24]:
# Comprobamos en la base de datos
query_comprobacion = """
                SELECT * FROM hoteles LIMIT 5
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(1, 'Hotel Monte Verde', False, 1.0, 1),
 (2, 'Hotel Brisas del Mar', False, 5.0, 1),
 (3, 'Hotel Camino del Sol', False, 1.0, 1),
 (4, 'Hotel Puerta del Cielo', False, 5.0, 1),
 (5, 'Hotel Encanto Real', False, 1.0, 1)]

In [25]:
tabla_clientes = df_hoteles[["id_cliente", "nombre", "apellido", "mail" ]]
tabla_clientes.head()

,id_cliente,nombre,apellido,mail
0,cliente1,Maite,Calatayud,maite.calatayud@example.com
1,cliente2,Tecla,Bonet,tecla.bonet@example.com
2,cliente3,Amílcar,Andrés,amílcar.andrés@example.com
3,cliente4,Joan,Vazquez,joan.vazquez@example.com
4,cliente5,Chelo,Flor,chelo.flor@example.com


In [26]:
tabla_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 5171
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  15000 non-null  object
 1   nombre      15000 non-null  object
 2   apellido    15000 non-null  object
 3   mail        15000 non-null  object
dtypes: object(4)
memory usage: 585.9+ KB


In [27]:
data_to_insert = []
for _, row in df_hoteles.iterrows():
    id_cliente = row["id_cliente"]
    nombre = row["nombre"]
    apellido = row["apellido"]
    mail = row["mail"]
    
    data_to_insert.append([id_cliente, nombre, apellido, mail])

In [28]:
insert_query = """
                INSERT INTO clientes(id_cliente, nombre, apellido, mail)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (id_cliente) DO NOTHING;
"""

In [29]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [30]:
# Comprobamos en la base de datos
query_comprobacion = """
                SELECT * FROM clientes LIMIT 5
"""
cur.execute(query_comprobacion)
cur.fetchall()

[('cliente1', 'Maite', 'Calatayud', 'maite.calatayud@example.com'),
 ('cliente2', 'Tecla', 'Bonet', 'tecla.bonet@example.com'),
 ('cliente3', 'Amílcar', 'Andrés', 'amílcar.andrés@example.com'),
 ('cliente4', 'Joan', 'Vazquez', 'joan.vazquez@example.com'),
 ('cliente5', 'Chelo', 'Flor', 'chelo.flor@example.com')]

In [31]:
tabla_reservas = df_hoteles[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche",
                              "nombre_hotel", "mail"]]
tabla_reservas.head()

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,nombre_hotel,mail
0,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,276.612381,Hotel Monte Verde,maite.calatayud@example.com
1,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,275.225921,Hotel Brisas del Mar,tecla.bonet@example.com
2,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,269.998444,Hotel Camino del Sol,amílcar.andrés@example.com
3,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,2025-02-08,2025-03-01,2025-03-02,280.151243,Hotel Puerta del Cielo,joan.vazquez@example.com
4,99c3dc4f-663c-45f7-849a-ac9313f3746a,2025-02-09,2025-03-01,2025-03-02,278.243996,Hotel Encanto Real,chelo.flor@example.com


In [32]:
tabla_reservas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 5171
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   fecha_reserva    15000 non-null  datetime64[ns]
 2   inicio_estancia  15000 non-null  datetime64[ns]
 3   final_estancia   15000 non-null  datetime64[ns]
 4   precio_noche     15000 non-null  float64       
 5   nombre_hotel     15000 non-null  object        
 6   mail             15000 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(3)
memory usage: 937.5+ KB


In [33]:
cur.execute("SELECT id_hotel, nombre_hotel FROM hoteles")  
hotel_list = cur.fetchall()  # Lista de tuplas
hotel_dict = {nombre.strip().lower(): id_h for id_h, nombre in hotel_list}
print(hotel_dict)  # Verificar que los datos se extrajeron correctamente

{'hotel monte verde': 9819, 'hotel brisas del mar': 9827, 'hotel camino del sol': 9814, 'hotel puerta del cielo': 9816, 'hotel encanto real': 9821, 'palacio del sol': 9818, 'hotel jardines del rey': 9826, 'hotel las estrellas': 9808, 'gran hotel madrid': 9823, 'hotel torre dorada': 9806, 'hotel palacio imperial': 9820, 'hotel luz de madrid': 9811, 'hotel los almendros': 9815, 'hotel sol y luna': 9792, 'hotel mirador real': 9777, 'hotel rincón sereno': 9825, 'hotel vista alegre': 9828, 'hotel costa azul': 9800, 'hotel maravilla real': 9812, 'ibis styles madrid prado': 14990, 'novotel madrid center': 15000, 'ibis madrid centro las ventas': 14985, 'novotel madrid city las ventas': 14999, 'ibis budget madrid vallecas': 14977, 'ibis madrid aeropuerto barajas': 14980, 'ibis madrid alcorcon tresaguas': 14997, 'ibis budget madrid aeropuerto': 14998, 'ibis madrid alcobendas': 14996, 'ibis budget madrid alcorcon móstoles': 14993}


In [34]:
cur.execute("SELECT id_cliente, mail FROM clientes")  
cliente_list = cur.fetchall()  # Lista de tuplas
cliente_dict = {nombre.strip().lower(): id_c for id_c, nombre in cliente_list}
print(cliente_dict)  # Verificar que los datos se extrajeron correctamente

{'maite.calatayud@example.com': 'cliente1', 'tecla.bonet@example.com': 'cliente2', 'amílcar.andrés@example.com': 'cliente3', 'joan.vazquez@example.com': 'cliente4', 'chelo.flor@example.com': 'cliente5', 'dolores.lobo@example.com': 'cliente6', 'noé.lobo@example.com': 'cliente7', 'omar.lamas@example.com': 'cliente8', 'ágata.pinedo@example.com': 'cliente9', 'julie.baños@example.com': 'cliente10', 'javier.mancebo@example.com': 'cliente11', 'rita.guerrero@example.com': 'cliente12', 'ricardo.lobo@example.com': 'cliente13', 'perlita.patiño@example.com': 'cliente14', 'pedro.gallego@example.com': 'cliente15', 'judith.bellido@example.com': 'cliente16', 'francisco javier.hierro@example.com': 'cliente17', 'reynaldo.lerma@example.com': 'cliente18', 'felicidad.gimenez@example.com': 'cliente19', 'pía.barba@example.com': 'cliente20', 'amílcar.moles@example.com': 'cliente21', 'blas.galván@example.com': 'cliente22', 'claudia.miranda@example.com': 'cliente23', 'fernanda.segarra@example.com': 'cliente24',

In [35]:
data_to_insert = []
for _, row in df_hoteles.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = row["precio_noche"]
    nombre_hotel = row["nombre_hotel"]
    mail = row["mail"]
    id_hotel = hotel_dict.get(nombre_hotel.strip().lower(), None)
    id_cliente = cliente_dict.get(mail.strip().lower(), None)
    
    data_to_insert.append([id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, 
                            id_hotel, id_cliente])

In [36]:
insert_query = """
                INSERT INTO reservas(id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, 
                            id_hotel, id_cliente)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [37]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [38]:
# Comprobamos en la base de datos
query_comprobacion = """
                SELECT * FROM reservas LIMIT 5
"""
cur.execute(query_comprobacion)
cur.fetchall()

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  datetime.date(2025, 2, 9),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  276.61238095238093,
  'cliente1',
  9819),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  datetime.date(2025, 2, 8),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  275.2259210526316,
  'cliente2',
  9827),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  datetime.date(2025, 2, 2),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  269.99844357976656,
  'cliente3',
  9814),
 ('ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7',
  datetime.date(2025, 2, 8),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  280.15124282982794,
  'cliente4',
  9816),
 ('99c3dc4f-663c-45f7-849a-ac9313f3746a',
  datetime.date(2025, 2, 9),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  278.24399606299215,
  'cliente5',
  9821)]

In [39]:
cur.close()
conn.close()